In [6]:
from utils import convert_json_to_sentence
import pandas as pd
from dataset import PromptDataset
from clip_linear import ClipLinearProbing
from torch.utils.data import random_split, DataLoader
import torch
import numpy as np  

In [2]:
data = pd.read_json('/home/guillermo/ssd/Github/BreastCATT/prompts/179.0.json')

convert_json_to_sentence(data)

'Patient is 93 years old, widow of white race, registered on 2013-08-27. Has eating habits rich in no fat. Their last menstruation was at 48 years old and menarche occurred at 14 years old. No information is provided for radiotherapy. The recorded body temperature is 34.50°C.'

In [3]:
dataset = PromptDataset('/home/guillermo/ssd/Github/BreastCATT/prompts', '/home/guillermo/ssd/Github/BreastCATT/patient_labels.json')

# Use DataLoader to load the data in batches
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = ClipLinearProbing(device)

In [28]:
# Calculate features

all_features = []
all_labels = []

for data in dataloader:
  prompt = data["prompt"][0]  # get the prompt string from the batch
  label = torch.tensor(data["label"][0]).float()
  
  with torch.no_grad():
      # model expects a list of prompt(s)
      text_features = model([prompt])
  all_features.append(text_features)
  all_labels.append(label.unsqueeze(0))

all_features = (torch.sigmoid(torch.cat(all_features).cpu()) > 0.5).float().numpy()
all_labels = torch.cat(all_labels).cpu().numpy()
print(all_features.shape, all_labels.shape)

/tmp/ipykernel_53273/2398609991.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(data["label"][0]).float()


(278, 1) (278,)


In [29]:
accuracy = np.mean((all_labels == all_features).astype(float)) * 100.
print(accuracy)

59.386160136638885


# problems

- max_length of CLIP is 77 (in this library idk if is the same with another version), so I have to truncate the prompt

# next steps

- linear regression (example here https://github.com/openai/CLIP)
- see if the context length can be increased (maybe with CLIP from Hugging Face or OpenCLIP)
- disminuir la longitud del prompt en caso de que lo de arriba no se pueda (sin truncarlo)